In [14]:
# Bring in all of our dependencies
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import gym
import math



In [15]:
# We should define the environment here and detremine the sizing so we can generalize our simple neural network
env    = gym.make('CartPole-v0')   # Choose our environment
tmp    = env.action_space.sample()
print(type(tmp))
action = np.array([tmp])
N      = len(action)               # Use this to generalize the input size for the NN
print(N)

[2016-11-26 17:05:46,645] Making new env: CartPole-v0


<class 'int'>
1


In [16]:
# Make a very simple neural network, reLU layers have been shown to train faster. We only have one hidden layer here, very simple
model = Sequential()
model.add(Dense(100, input_shape=((4,)), activation='relu',init='uniform'))
model.add(Dense(100, activation='relu',init='uniform'))
model.add(Dense(2))

In [17]:
# Build the model for training, we can tweak our loss function and optimizer for different environments
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# Initialize replay memory
#repMem = np.empty((0,4))
gamma = 0.8
epsUpdate = 0.1
maxMem = 200
obs1 = env.reset()
action = env.action_space.sample()
obs2, reward, done, info = env.step(action)
repMem = np.array([obs1,action,reward,obs2,done])

#Start monigtoring
env.monitor.start('/tmp/cartpole-experiment-1')

#repMem = [1,2,3,4]
# Run episodes tally results
for i_episode in range(500):
    obs1 = env.reset()
    epsilon = epsUpdate * i_episode
    rTally = np.array([0])
    loss = 0.
    
    for t in range(300):
        env.render()
        obs2 = obs1
        
        # Add action selection by max output of network over time, update epsilon slowly
        if np.random.uniform(0,1,1) >= (epsilon):
            action = env.action_space.sample()                    # this is a random action
        else:
            action = int(np.argmax(model.predict(obs2[np.newaxis,])[0]))  
            
        obs1, reward, done, info = env.step(action)    # apply the action to the model and check the outputs
        
        # Quit early if too many steps
        if t == 250:
            done = True
            
        repMem = np.column_stack((repMem, np.array([obs2,action,reward,obs1,done])))
        
        if repMem.shape[1] > maxMem:
            np.delete(repMem,repMem.shape[1]-1,1)
            
        if repMem.shape[1] > 10:
            P = max(float(1),math.floor(repMem.shape[1]/5))
        else:
            P = int(1)

        # Get random batch
        memBatch = repMem[:,np.random.choice(repMem.shape[1], int(P), replace=False)]
        
            
        # Put Q-Learning stuff here to start...
        # Q[s,a] = Q[s,a] + a(r + y max_a'(Q[s',a'] - Q[s,a]))
        # Get targets
        tgt = np.zeros((P,2))
        
        for batchNum in range(P):
            
            tmp0 = memBatch[0,batchNum]
            tmp0 = tmp0[np.newaxis,]
            tgt[batchNum,:] = model.predict(tmp0)[0]
            
            tmp = memBatch[3,batchNum]
            tmp = tmp[np.newaxis,]
            QVal = np.max(model.predict(tmp)[0])
            
            tmpReward = memBatch[2,batchNum]
            tmpA      = memBatch[1,batchNum]
            
            tmpDone = memBatch[4,batchNum]
            if tmpDone:
                tgt[batchNum,tmpA] = tmpReward
            else:
                tgt[batchNum,tmpA] = tmpReward + (gamma * QVal)                            
        # End batch for
        
       
        # Train              
        if memBatch[0,:].shape[0] == 1:
            tmp = memBatch[0,]
            tmp = tmp[0]
            tmp = tmp[np.newaxis,]
        else:
            tmp = np.vstack(memBatch[0,:])
        
        loss += model.train_on_batch(tmp, tgt)
        #print(loss)
            
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
            
        env.monitor.close()

[2016-11-26 17:13:21,327] Creating monitor directory /tmp/cartpole-experiment-1
[2016-11-26 17:13:21,339] Starting new video recorder writing to C:\tmp\cartpole-experiment-1\openaigym.video.6.5616.video000000.mp4
[2016-11-26 17:13:21,472] Finished writing results. You can upload them to the scoreboard via gym.upload('C:\\tmp\\cartpole-experiment-1')


Episode finished after 26 timesteps
Episode finished after 12 timesteps
Episode finished after 11 timesteps
Episode finished after 49 timesteps
Episode finished after 66 timesteps
Episode finished after 20 timesteps
Episode finished after 10 timesteps
Episode finished after 18 timesteps
Episode finished after 10 timesteps
Episode finished after 35 timesteps
Episode finished after 28 timesteps
Episode finished after 55 timesteps
Episode finished after 13 timesteps
Episode finished after 34 timesteps
Episode finished after 107 timesteps
Episode finished after 145 timesteps
